# ADS 509 Module 3: Group Comparison 

The task of comparing two groups of text is fundamental to textual analysis. There are innumerable applications: survey respondents from different segments of customers, speeches by different political parties, words used in Tweets by different constituencies, etc. In this assignment you will build code to effect comparisons between groups of text data, using the ideas learned in reading and lecture.

This assignment asks you to analyze the lyrics and Twitter descriptions for the two artists you selected in Module 1. If the results from that pull were not to your liking, you are welcome to use the zipped data from the “Assignment Materials” section. Specifically, you are asked to do the following: 

* Read in the data, normalize the text, and tokenize it. When you tokenize your Twitter descriptions, keep hashtags and emojis in your token set. 
* Calculate descriptive statistics on the two sets of lyrics and compare the results. 
* For each of the four corpora, find the words that are unique to that corpus. 
* Build word clouds for all four corpora. 

Each one of the analyses has a section dedicated to it below. Before beginning the analysis there is a section for you to read in the data and do your cleaning (tokenization and normalization). 


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


In [2]:
!pip install wordcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 4.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
# Import necessary packages
import os
import re
import emoji
import pandas as pd

from collections import Counter, defaultdict
from nltk.corpus import stopwords
from string import punctuation
from wordcloud import WordCloud 

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [4]:
# Use this space for any additional import statements you need
from nltk.tokenize import word_tokenize
import emoji
import re

In [5]:
# Place any addtional functions or constants you need here. 

# Some punctuation variations
punctuation = set(punctuation) # speeds up comparison
tw_punct = punctuation - {"#"}

# Stopwords
sw = stopwords.words("english")

# Two useful regex
whitespace_pattern = re.compile(r"\s+")
hashtag_pattern = re.compile(r"^#[0-9a-zA-Z]+")

# It's handy to have a full set of emojis
all_language_emojis = set()

for country in emoji.EMOJI_DATA : 
    for em in emoji.EMOJI_DATA[country] : 
        all_language_emojis.add(em)

# and now our functions
def descriptive_stats(tokens, num_tokens = 5, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity, and num_tokens most common
        tokens. Return a list of 
    """

    # Place your Module 2 solution here
    
    # Convert tokens to a list to it won't print just single letters
    tokens_list = tokens.split()
    
    if verbose :        
        print(f"There are {len(tokens)} tokens in the data.")
        print(f"There are {len(set(tokens))} unique tokens in the data.")
        print(f"There are {len(''.join(tokens))} characters in the data.")
        print(f"The lexical diversity is {len(set(tokens))/len(tokens):.3f} in the data.")
    
        # print the five most common tokens
        counts = Counter(tokens_list)
        
        if num_tokens > 0:
            print(counts.most_common(num_tokens))
        
    return([len(tokens), 
            len(set(tokens)),
            len("".join(tokens)),
            len(set(tokens))/len(tokens)])    
    
    return(0)

def contains_emoji(s):
    
    s = str(s)
    emojis = [ch for ch in s if emoji.is_emoji(ch)]

    return(len(emojis) > 0)


def remove_stop(tokens) :
    # modify this function to remove stopwords
    return(tokens)
 
def remove_punctuation(text, punct_set=tw_punct) : 
    return("".join([ch for ch in text if ch not in punct_set]))

def tokenize(text) : 
    """ Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. """
    
    # modify this function to return tokens
    tokens = text.split()
    tokens = [token.strip() for token in tokens]
    return tokens

def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens)

## Data Ingestion

Use this section to ingest your data into the data structures you plan to use. Typically this will be a dictionary or a pandas DataFrame.

In [6]:
# Feel free to use the below cells as an example or read in the data in a way you prefer
data_location = "/users/christophergarcia/Desktop/ads-tm-api-scrape/"

twitter_folder = "twitter/"
lyrics_folder = "lyrics/"

artist_files = {'cher':'cher_followers_data.txt',
                'robyn':'robynkonichiwa_followers_data.txt'}

In [7]:
twitter_data = pd.read_csv(data_location + twitter_folder + artist_files['cher'],
                           sep="\t",
                           quoting=3)

twitter_data['artist'] = "cher"

In [8]:
twitter_data_2 = pd.read_csv(data_location + twitter_folder + artist_files['robyn'],
                             sep="\t",
                             quoting=3)
twitter_data_2['artist'] = "robyn"

twitter_data = pd.concat([
    twitter_data,twitter_data_2])
    
del(twitter_data_2)

In [9]:
twitter_data

,screen_name,name,id,location,followers_count,friends_count,description,artist
0,hsmcnp,Country Girl,35152213,NaN,1302,1014,NaN,cher
1,horrormomy,Jeny,742153090850164742,Earth,81,514,𝙿𝚛𝚘𝚞𝚍 𝚜𝚞𝚙𝚙𝚘𝚛𝚝𝚎𝚛 𝚘𝚏 𝚖𝚎𝚜𝚜𝚢 𝚋𝚞𝚗𝚜 & 𝚕𝚎𝚐𝚐𝚒𝚗𝚐𝚜,cher
2,anju79990584,anju,1496463006451974150,NaN,13,140,163㎝／愛かっぷ💜26歳🍒 工〇好きな女の子💓 フォローしてくれたらDMします🧡,cher
3,gallionjenna,J,3366479914,NaN,752,556,csu,cher
4,bcscomm,bcscomm,83915043,"Washington, DC",888,2891,Writer @Washinformer @SpelmanCollege alumna #D...,cher
...,...,...,...,...,...,...,...,...
358367,jakegiles,Jake Giles,19725102,LA,7690,2165,"singer of songs, type 1 diabetic, tired $jakel...",robyn
358368,axelbluhme,Axel Bluhme,19573759,DK,238,1565,Dadx2/ Con-Arch/ Photographer/ DK #stemgrønnes...,robyn
358369,RayRayRay27,Rachael :),19919217,Oldham,762,1479,A year to change a life is still a year ✨😌,robyn
358370,bhandberg,Ben Handberg,12642462,Sydney,432,593,Head of Consumer - Mango. Made in Melbourne. R...,robyn


In [10]:
# read in the lyrics here
lyrics_data = []

for artist_folder in os.listdir(data_location + lyrics_folder):
    if os.path.isdir(data_location + lyrics_folder + artist_folder):
        for lyrics_file in os.listdir(data_location + lyrics_folder + artist_folder):
            artist, song = lyrics_file.split("_")
            song = song.replace(".txt", "")
            lyrics = open(data_location + lyrics_folder + artist_folder + "/" + lyrics_file).read()
            lyrics_data.append([artist, song, lyrics])

# Create a DataFrame from the lyrics data
lyrics_data = pd.DataFrame(lyrics_data, columns=['artist', 'song', 'lyrics'])

In [11]:
lyrics_data

,artist,song,lyrics
0,robyn,includemeout,"""Include Me Out""\n\n\n\nIt is really very simp..."
1,robyn,electric,"""Electric""\n\n\n\nElectric...\n\nIt's electric..."
2,robyn,beach2k20,"""Beach 2K20""\n\n\n\n(So you wanna go out?\nHow..."
3,robyn,lovekills,"""Love Kills""\n\n\n\nIf you're looking for love..."
4,robyn,timemachine,"""Time Machine""\n\n\n\nHey, what did I do?\nCan..."
...,...,...,...
256,cher,chastityssongbandofthieves,"""Chastity's Song (Band Of Thieves)""\n\n\n\nWin..."
257,cher,thisgodforsakenday,"""This God-Forsaken Day""\n\n\n\nRead your note ..."
258,cher,time,"""Time""\n\n\n\nSome people run \nSome people cr..."
259,cher,dreambaby,"""Dream Baby""\n\n\n\nI found the boy for me\nHe..."


## Tokenization and Normalization

In this next section, tokenize and normalize your data. We recommend the following cleaning. 

**Lyrics** 

* Remove song titles
* Casefold to lowercase
* Remove stopwords (optional)
* Remove punctuation
* Split on whitespace

Removal of stopwords is up to you. Your descriptive statistic comparison will be different if you include stopwords, though TF-IDF should still find interesting features for you. Note that we remove stopwords before removing punctuation because the stopword set includes punctuation.

**Twitter Descriptions** 

* Casefold to lowercase
* Remove stopwords
* Remove punctuation other than emojis or hashtags
* Split on whitespace

Removing stopwords seems sensible for the Twitter description data. Remember to leave in emojis and hashtags, since you analyze those. 

In [12]:
# apply the `pipeline` techniques from BTAP Ch 1 or 5
my_pipeline = [str.lower, remove_punctuation, tokenize, remove_stop]

lyrics_data["tokens"] = lyrics_data["lyrics"].apply(prepare,pipeline=my_pipeline)
lyrics_data["num_tokens"] = lyrics_data["tokens"].map(len) 

twitter_data["tokens"] = twitter_data["description"].apply(prepare,pipeline=my_pipeline)
twitter_data["num_tokens"] = twitter_data["tokens"].map(len) 

In [13]:
twitter_data['has_emoji'] = twitter_data["description"].apply(contains_emoji)

Let's take a quick look at some descriptions with emojis.

In [14]:
twitter_data[twitter_data.has_emoji].sample(10)[["artist","description","tokens"]]

,artist,description,tokens
3783606,cher,"Puppies, Chakras, Crystals, YouTube, Twitch, B...","[puppies, chakras, crystals, youtube, twitch, ..."
74415,cher,Keep it simple🤟😎,"[keep, it, simple🤟😎]"
166064,robyn,Audio Wizard(Cubase & Wwise)🎧Gamer geek since ...,"[audio, wizardcubase, wwise🎧gamer, geek, since..."
1111387,cher,👽🔰I Don't Want To Explain My Self😜😀.Because Ev...,"[👽🔰i, dont, want, to, explain, my, self😜😀becau..."
238928,cher,sc: davidchapa2004🤠||FHS||’23ll Varsity Cheer ...,"[sc, davidchapa2004🤠fhs’23ll, varsity, cheer, ..."
2118289,cher,"Stay desi,Stay weird🤪","[stay, desistay, weird🤪]"
96976,robyn,eh 🥀💔,"[eh, 🥀💔]"
1465764,cher,just a happy girl ☺️🧡,"[just, a, happy, girl, ☺️🧡]"
906514,cher,"Do not judge me, you know my name, not my stor...","[do, not, judge, me, you, know, my, name, not,..."
3644832,cher,#100sangresinaloense ✌️,"[#100sangresinaloense, ✌️]"


With the data processed, we can now start work on the assignment questions. 

Q: What is one area of improvement to your tokenization that you could theoretically carry out? (No need to actually do it; let's not make perfect the enemy of good enough.)

A: 

## Calculate descriptive statistics on the two sets of lyrics and compare the results. 


In [15]:
# your code here
def compare_texts(corpus_1, corpus_2, num_words = 10, ratio_cutoff = 5):
    results = dict()
    
    # Calculate descriptive statistics for corpus 1
    stats_corpus_1 = descriptive_stats(corpus_1)
    results['corpus_1'] = stats_corpus_1

    # Calculate descriptive statistics for corpus 2
    stats_corpus_2 = descriptive_stats(corpus_2)
    results['corpus_2'] = stats_corpus_2

    return results

In [16]:
lyrics_data

,artist,song,lyrics,tokens,num_tokens
0,robyn,includemeout,"""Include Me Out""\n\n\n\nIt is really very simp...","[include, me, out, it, is, really, very, simpl...",453
1,robyn,electric,"""Electric""\n\n\n\nElectric...\n\nIt's electric...","[electric, electric, its, electric, its, a, na...",253
2,robyn,beach2k20,"""Beach 2K20""\n\n\n\n(So you wanna go out?\nHow...","[beach, 2k20, so, you, wanna, go, out, how, yo...",291
3,robyn,lovekills,"""Love Kills""\n\n\n\nIf you're looking for love...","[love, kills, if, youre, looking, for, love, g...",413
4,robyn,timemachine,"""Time Machine""\n\n\n\nHey, what did I do?\nCan...","[time, machine, hey, what, did, i, do, cant, b...",230
...,...,...,...,...,...
256,cher,chastityssongbandofthieves,"""Chastity's Song (Band Of Thieves)""\n\n\n\nWin...","[chastitys, song, band, of, thieves, wind, up,...",236
257,cher,thisgodforsakenday,"""This God-Forsaken Day""\n\n\n\nRead your note ...","[this, godforsaken, day, read, your, note, thi...",195
258,cher,time,"""Time""\n\n\n\nSome people run \nSome people cr...","[time, some, people, run, some, people, crawl,...",127
259,cher,dreambaby,"""Dream Baby""\n\n\n\nI found the boy for me\nHe...","[dream, baby, i, found, the, boy, for, me, hes...",164


In [17]:
# Naming the corpus name for lyrics data of both artists
corpus1 = lyrics_data[lyrics_data['artist'] == 'cher']['lyrics']
corpus2 = lyrics_data[lyrics_data['artist'] == 'robyn']['lyrics']

In [18]:
# Comparisons on descriptive statistics on the two sets of lyrics
results = compare_texts(' '.join(corpus1), ' '.join(corpus2), num_words=5)
print(results)

There are 146053 tokens in the data.
There are 73 unique tokens in the data.
There are 146053 characters in the data.
The lexical diversity is 0.000 in the data.
[('I', 1174), ('the', 883), ('you', 853), ('to', 610), ('me', 544)]
There are 147880 tokens in the data.
There are 90 unique tokens in the data.
There are 147880 characters in the data.
The lexical diversity is 0.001 in the data.
[('you', 1216), ('I', 831), ('the', 811), ('me', 659), ('to', 640)]
{'corpus_1': [146053, 73, 146053, 0.0004998185590162475], 'corpus_2': [147880, 90, 147880, 0.0006086015688395997]}


Q: what observations do you make about these data? 

A: I noticed that there are is a large amount of tokens for the Cher's lyrics as opposed to Robyn's lyrics. Cher's most common lyrics include: "I", "you", "the", "to", "a."


## Find tokens uniquely related to a corpus

Typically we would use TF-IDF to find unique tokens in documents. Unfortunately, we either have too few documents (if we view each data source as a single document) or too many (if we view each description as a separate document). In the latter case, our problem will be that descriptions tend to be short, so our matrix would be too sparse to support analysis. 

To avoid these problems, we will create a custom statistic to identify words that are uniquely related to each corpus. The idea is to find words that occur often in one corpus and infrequently in the other(s). Since corpora can be of different lengths, we will focus on the _concentration_ of tokens within a corpus. "Concentration" is simply the count of the token divided by the total corpus length. For instance, if a corpus had length 100,000 and a word appeared 1,000 times, then the concentration would be $\frac{1000}{100000} = 0.01$. If the same token had a concentration of $0.005$ in another corpus, then the concentration ratio would be $\frac{0.01}{0.005} = 2$. Very rare words can easily create infinite ratios, so you will also add a cutoff to your code so that a token must appear at least $n$ times for you to return it. 

An example of these calculations can be found in [this spreadsheet](https://docs.google.com/spreadsheets/d/1P87fkyslJhqXFnfYezNYrDrXp_GS8gwSATsZymv-9ms). Please don't hesitate to ask questions if this is confusing. 

In this section find 10 tokens for each of your four corpora that meet the following criteria: 

1. The token appears at least `n` times in all corpora
1. The tokens are in the top 10 for the highest ratio of appearances in a given corpora vs appearances in other corpora.

You will choose a cutoff for yourself based on the side of the corpus you're working with. If you're working with the Robyn-Cher corpora provided, `n=5` seems to perform reasonably well.

In [26]:
# Using the corpus_1 and corpus_2 frm the previous section
corpus1 = lyrics_data[lyrics_data['artist'] == 'cher']['lyrics']
corpus2 = lyrics_data[lyrics_data['artist'] == 'robyn']['lyrics']

# Putting both corpus in a list for future use
corpuslist = [corpus1, corpus2]

In [27]:
# your code here
# The idea is to compare the lyrics of two singers (Cher's lyrics vs Robyn's lyrics)

# Finding the token that appears at least n times in all corpora
def findTokens_minCount(corpuslist, min_count):
    tokens_minCount = []
    
    for corpus in corpuslist:
        for song in corpus:
            tokens = word_tokenize(song)
            token_counts = Counter(tokens)
            
            for token, count in token_counts.items():
                if count >= min_count and token not in tokens_minCount:
                    tokens_minCount.append(token)
                    
                    if len(tokens_minCount) == 10:
                        return tokens_minCount
    
    return tokens_minCount

In [28]:
min_count = 5

tokens_minCount = findTokens_minCount(corpuslist, min_count)

print("Tokens that appear at least", min_count, "times in all corpora:")
print(tokens_minCount)

Tokens that appear at least 5 times in all corpora:
['I', "'ll", 'be', 'can', 'you', 'and', 'stay', 'with', 'me', 'Pirate']


Q: What are some observations about the top tokens? Do you notice any interesting items on the list? 

A: 

## Build word clouds for all four corpora. 

For building wordclouds, we'll follow exactly the code of the text. The code in this section can be found [here](https://github.com/blueprints-for-text-analytics-python/blueprints-text/blob/master/ch01/First_Insights.ipynb). If you haven't already, you should absolutely clone the repository that accompanies the book. 


In [25]:
from matplotlib import pyplot as plt

def wordcloud(word_freq, title=None, max_words=200, stopwords=None):

    wc = WordCloud(width=800, height=400, 
                   background_color= "black", colormap="Paired", 
                   max_font_size=150, max_words=max_words)
    
    # convert data frame into dict
    if type(word_freq) == pd.Series:
        counter = Counter(word_freq.fillna(0).to_dict())
    else:
        counter = word_freq

    # filter stop words in frequency counter
    if stopwords is not None:
        counter = {token:freq for (token, freq) in counter.items() 
                              if token not in stopwords}
    wc.generate_from_frequencies(counter)
 
    plt.title(title) 

    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    
    
def count_words(df, column='tokens', preprocess=None, min_freq=2):

    # process tokens and update counter
    def update(doc):
        tokens = doc if preprocess is None else preprocess(doc)
        counter.update(tokens)

    # create counter and run through all data
    counter = Counter()
    df[column].map(update)

    # transform counter into data frame
    freq_df = pd.DataFrame.from_dict(counter, orient='index', columns=['freq'])
    freq_df = freq_df.query('freq >= @min_freq')
    freq_df.index.name = 'token'
    
    return freq_df.sort_values('freq', ascending=False)

In [24]:
#wordcloud(cher_lyrics_count['freq'], title = 'Cher Lyrics', max_words = 50)

Q: What observations do you have about these (relatively straightforward) wordclouds? 

A: 